In [2]:
import datetime, time, math, os, json, shutil, asyncio, networkx as nx
from utils.gitutils import create_pull_request, clone_repo, create_branch, wipe_repo, prepare_repo
from utils.agent import Agent, GenerationConfig, Interaction, Team
from utils.stringutils import arr_from_sep_string, extract_markdown_blocks, markdown_to_dict
from utils.filetreeutils import FileTree, write_file_tree
from utils.listutils import flatten
from utils.frameworkutils import DartAnalyzer
from dotenv import load_dotenv
from utils.graphutils import loose_level_order, collapsed_level_order
from utils.frameworkutils import Framework

In [3]:
load_dotenv()  # Load environment variables from .env

GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
API_KEY = os.getenv('API_KEY')

In [4]:
repo_url = 'https://github.com/6165-MSET-CuttleFish/TeamTrack'
source = Framework.FLUTTER
feature = f"""# Name: Organizations
# Description: Organizations are a way to group users together. They can be used to automatically share events with the members of the organization whenever they are uploaded to the cloud. \
Organizations consist of a name, team number, and a backend-generated unique id. They can also have a profile picture, description, and location. \
Members can also opt to be publicly listed as a member of the organization or not. \
"""

In [5]:
# Clone the repo and make a new branch
repo = clone_repo(repo_url)
base_branch = repo.active_branch.name
created_branch = f"modification-{base_branch}"
local_repo_path = str(repo.working_dir)
working_dir_path = f'{local_repo_path}\\{source.get_working_dir()}'

In [6]:
source_swe_summarizer = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_analyzer",
    generation_config=GenerationConfig(temperature=0.7),
    system_prompt=f'''You are a {source} software engineer. Your job is to summarize the functionality of provided code files. Structure your response as follows:

# filename.ext:

$7 sentence summary of the functionality/contents of the file$
'''
)

pm = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="pm",
    generation_config=GenerationConfig(temperature=0.7),
    system_prompt=f"""You are a high-level technical project manager tasked with the project of adding functionality to a {source} repository. For each feature request, you must provided a numbered list of tasks that need to be completed. \
Format your response as follows: \
1. $ACTION$
2. $ACTION$
------------------
You may choose ACTIONS from the following list:
- ANALYZE $FILE_PATH$ (Analyze the provided file)
- WRITE $PROMPT$ (Write code to fulfill the prompt)
- EDIT $FILE_PATH$ | $PROMPT$ (Edit the provided file to fulfill the prompt)
"""
)

source_swe_coder = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_coder",
    generation_config=GenerationConfig(temperature=0.7),
    system_prompt=f'''You are a {source} software engineer. Your job is to write code to implement a specific feature. Respond with only code.'''
)

team = Team(source_swe_summarizer, pm, source_swe_coder)

In [7]:
analyzer = source.get_analyzer(working_dir_path)

source_dependency_graph = analyzer.buildDependencyGraph()

In [8]:
# loose level order
eval_order = loose_level_order(source_dependency_graph)[::-1]
for level in eval_order:
    print(level)
print(f"Number of levels: {len(eval_order)}")

['models\\StatConfig.dart', 'models\\AppModel.dart']
['models\\GameModel.dart']
['models\\ScoreModel.dart']
['functions\\Extensions.dart', 'components\\misc\\PlatformGraphics.dart']
['components\\statistics\\BarGraph.dart', 'components\\users\\PFP.dart']
['functions\\Statistics.dart', 'providers\\Auth.dart', 'components\\scores\\ScoreSummary.dart', 'components\\users\\UsersRow.dart', 'components\\scores\\Incrementor.dart']
['views\\home\\match\\MatchView.dart']
['components\\statistics\\PercentChange.dart', 'functions\\Functions.dart']
['components\\scores\\ScoreRangeSummary.dart', 'models\\Change.dart', 'components\\scores\\ScoreTimeline.dart']
['components\\statistics\\CheckList.dart', 'components\\misc\\EmptyList.dart', 'views\\home\\change\\ChangeConfig.dart', 'components\\misc\\CardView.dart', 'views\\home\\change\\ChangeRow.dart', 'views\\home\\match\\MatchRow.dart', 'views\\home\\match\\MatchConfig.dart', 'components\\scores\\ScoringElementStats.dart', 'views\\home\\match\\Examp

In [9]:
(source_file_tree := FileTree.from_dir(working_dir_path))

├── api\
│   ├── README.MD
├── components\
│   ├── misc\
│   │   ├── CardView.dart
│   │   ├── Collapsible.dart
│   │   ├── EmptyList.dart
│   │   ├── InfoPills.dart
│   │   ├── PlatformGraphics.dart
│   ├── scores\
│   │   ├── Incrementor.dart
│   │   ├── ScoreCard.dart
│   │   ├── ScoreRangeSummary.dart
│   │   ├── ScoreSummary.dart
│   │   ├── ScoreTimeline.dart
│   │   ├── ScoringElementStats.dart
│   ├── statistics\
│   │   ├── BarGraph.dart
│   │   ├── CheckList.dart
│   │   ├── PercentChange.dart
│   ├── users\
│   │   ├── PFP.dart
│   │   ├── UsersRow.dart
│   ├── AutonomousDrawingTool.dart
├── functions\
│   ├── APIMethods.dart
│   ├── Extensions.dart
│   ├── Functions.dart
│   ├── GPTModel.dart
│   ├── ResponseModel.dart
│   ├── Statistics.dart
├── models\
│   ├── AppModel.dart
│   ├── Change.dart
│   ├── GameModel.dart
│   ├── GPTModel.dart
│   ├── ScoreModel.dart
│   ├── StatConfig.dart
├── providers\
│   ├── Auth.dart
│   ├── PushNotifications.dart
│   ├── Theme.dart
│   ├

In [11]:
# Summarize the original repo
from tqdm import tqdm

async def summarize_group(group):
    async def summarize(node):
        name = source_file_tree.nodes[node]['name']
        content = source_file_tree.nodes[node]['content']
        message = f"{name}\n```\n{content}\n```"
        return await team.async_chat_with_agent(
            agent_name='source_swe_analyzer',
            message=message,
            context_keys=[f"summary_{neighbor}" for neighbor in source_dependency_graph[node]],
            save_keys=[f"summary_{node}", "all"],
            prompt_title=f"Summary of {node}"
            )
    tasks = [summarize(node) for node in group]
    responses = await asyncio.gather(*tasks)
    for i, response in enumerate(responses):
        source_file_tree.nodes[group[i]]["summary"] = response

for level in tqdm(eval_order):
    await summarize_group(level)
    time.sleep(0.5)

100%|██████████| 19/19 [01:23<00:00,  4.41s/it]


In [12]:
prompt = f"""This is the file tree for the original {source} repo:
```
{source.get_working_dir()}\\
{source_file_tree}
```
Given the prior context, summarize the functionality of the entire repository succinctly.
"""
team.chat_with_agent('source_swe_analyzer', prompt, context_keys=["all"], save_keys=["all"], prompt_title=f"""Repository Summary.
File Tree provided:
```
{source.get_working_dir()}\\
{source_file_tree}
```
""")

'The Framework.FLUTTER repository implements a comprehensive mobile application for managing and analyzing data related to FTC robotics competitions. \n\n**Core Features:**\n\n- **Event Management:**\n    - Create, edit, and delete local events.\n    - Share events with other users and manage their permissions.\n    - Import match data from images using image processing and text recognition.\n    - Fetch and display data from the The Orange Alliance (TOA) API.\n\n- **Team Management:**\n    - View and analyze team performance based on various statistics.\n    - Sort teams based on different criteria like score, rank, and operational mode.\n    - Generate AI-driven recommendations for alliance selection using a GPT model.\n    - Draw and save autonomous paths for teams.\n\n- **Match Management:**\n    - View and manage match details, including scores, alliances, and active users.\n    - Create new matches and delete existing ones.\n    - Configure match settings like penalties and allia

In [13]:
curr_string = '\n'.join([message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict()])
(curr_tokens := pm.model.count_tokens(curr_string).total_tokens)

20646

In [14]:
would_be_string = '\n'.join([source_file_tree.nodes[node]['content'] for node in source_dependency_graph.nodes])
(would_be_tokens := pm.model.count_tokens(would_be_string).total_tokens)

123056

In [15]:
(percent_tokens_reduction := 1 - (curr_tokens / would_be_tokens))

0.8322227278637369

## Context Caching

In [16]:
pm.model.count_tokens('\n'.join(message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict())).total_tokens

20646

In [17]:
# Cache the context threads
factor = 32768 / pm.model.count_tokens('\n'.join(message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict())).total_tokens
# eg. if factor is 3.2, round to 4
factor = math.ceil(factor)
team.context_threads['all'] *= factor
pm.model.count_tokens('\n'.join(message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict())).total_tokens

41164

In [18]:
pm.cache_context(team.context_threads['all'], ttl=datetime.timedelta(minutes=5))

In [19]:
# Create new file tree
prompt = f'''
You are to build the following feature:
{feature}
'''
response = pm.chat(prompt)

In [20]:
response

'Okay, here\'s a breakdown of how to implement the "Organizations" feature, along with the necessary code snippets and explanations:\n\n**1. Data Model:**\n\n```dart\nclass Organization {\n  String id;\n  String name;\n  int teamNumber;\n  String? profilePictureUrl;\n  String? description;\n  String? location;\n\n  Organization({\n    required this.id,\n    required this.name,\n    required this.teamNumber,\n    this.profilePictureUrl,\n    this.description,\n    this.location,\n  });\n\n  // Constructor to create an Organization from a Map (for Firebase)\n  factory Organization.fromJson(Map<String, dynamic> json) {\n    return Organization(\n      id: json[\'id\'],\n      name: json[\'name\'],\n      teamNumber: json[\'teamNumber\'],\n      profilePictureUrl: json[\'profilePictureUrl\'],\n      description: json[\'description\'],\n      location: json[\'location\'],\n    );\n  }\n\n  // Method to convert Organization to Map (for Firebase)\n  Map<String, dynamic> toJson() {\n    return